In [1]:
#r "nuget: Jinaga.UnitTest"
#r "nuget: Jinaga.Graphviz"
using Jinaga;
using Jinaga.UnitTest;
using Jinaga.Graphviz;

var j = JinagaTest.Create();

Installed Packages Jinaga.Graphviz, 0.4.0 Jinaga.UnitTest, 0.4.0

In [2]:
[FactType("Restaurant.Server")]
record Server(string identifier) {}

[FactType("Restaurant.Table")]
record Table(int tableNumber) {}

[FactType("Restaurant.TableAssignment")]
record TableAssignment(Server server, Table table) {}

[FactType("Restaurant.Party")]
record Party(DateTime arrival) {}

[FactType("Restaurant.PartySeated")]
record PartySeated(Party party, Table table) {}


Jinaga.Graphviz.Renderer.RenderTypes(typeof(TableAssignment), typeof(PartySeated))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 8.0.2 (20230410.1723)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Restaurant.TableAssignment -->
 
 Restaurant.TableAssignment 
 
 Restaurant.TableAssignment 
 
<!-- Restaurant.Server -->
 
 Restaurant.Server 
 
 Restaurant.Server 
 
<!-- Restaurant.TableAssignment->Restaurant.Server -->
 
 Restaurant.TableAssignment->Restaurant.Server 
 
 
 server 
 
<!-- Restaurant.Table -->
 
 Restaurant.Table 
 
 Restaurant.Table 
 
<!-- Restaurant.TableAssignment->Restaurant.Table -->
 
 Restaurant.TableAssignment->Restaurant.Table 
 
 
 table 
 
<!-- Restaurant.PartySeated -->
 
 Restaurant.PartySeated 
 
 Restaurant.PartySeated 
 
<!-- Restaurant.PartySeated->Restaurant.Table -->
 
 Restaurant.PartySeated->Restaurant.Table 
 
 
 table 
 
<!-- Restaurant.Party -->
 
 Restaurant.Party 
 
 Restaurant.Party 
 
<!-- Restaurant.PartySeated->Restaurant.Party -->
 
 Restaurant.PartySeated->Restaurant.Party 
 
 
 party

We want to find out which parties are served by which server.
In this case we have 2 valley-facts (tableAssignment and partySeated). So which one to start from ?
Start from both and link them together with a "where" clause.


In [6]:
var partiesServedBy_1 = Given<Server>.Match((server, facts) => 
    from tableAssignment in facts.OfType<TableAssignment>()
        where tableAssignment.server == server
    from partySeated in facts.OfType<PartySeated>()
        where partySeated.table == tableAssignment.table
    from party in facts.OfType<Party>()
        where party == partySeated.party
    select party
);

partiesServedBy_1.ToString()

(server: Restaurant.Server) {
    tableAssignment: Restaurant.TableAssignment [
        tableAssignment->server: Restaurant.Server = server
    ]
    partySeated: Restaurant.PartySeated [
        partySeated->table: Restaurant.Table = tableAssignment->table: Restaurant.Table
    ]
    party: Restaurant.Party [
        party = partySeated->party: Restaurant.Party
    ]
} => party


Note that OfType<T> has an overload which allows you to omit the "where" clause.....

In [7]:
var partiesServedBy_2 = Given<Server>.Match((server, facts) => 
    from tableAssignment in facts.OfType<TableAssignment>(
        tableAssignment => tableAssignment.server == server)
    from partySeated in facts.OfType<PartySeated>(
        partySeated => partySeated.table == tableAssignment.table)
    from party in facts.OfType<Party>(
        party => party == partySeated.party)
    select party
);

partiesServedBy_2.ToString()

(server: Restaurant.Server) {
    tableAssignment: Restaurant.TableAssignment [
        tableAssignment->server: Restaurant.Server = server
    ]
    partySeated: Restaurant.PartySeated [
        partySeated->table: Restaurant.Table = tableAssignment->table: Restaurant.Table
    ]
    party: Restaurant.Party [
        party = partySeated->party: Restaurant.Party
    ]
} => party
